In [1]:
###############################################
# Rebrov reactor batch script
# Ting-Chen Lee and Chris Blais
# Northeastern University
# runs through all reactor conditions
###############################################

import numpy as np
import time
import cantera as ct
from matplotlib import pyplot as plt
import csv
import math
import os
import sys
import re
import itertools
import logging
from collections import defaultdict
import git
import json

from rmgpy.molecule import Molecule
from rmgpy.data.base import Database

def save_pictures(git_path="", species_path="", overwrite=False):
    """
    Save a folder full of molecule pictures, needed for the pretty dot files.

    Saves them in the results directory, in a subfolder "species_pictures".
    Unless you set overwrite=True, it'll leave alone files that are
    already there.
    """
    dictionary_filename = git_path + "/base/chemkin/species_dictionary.txt"
    specs = Database().get_species(dictionary_filename, resonance=False)

    images_dir = os.path.join(species_path)
    os.makedirs(images_dir, exist_ok=True)
    for name, species in specs.items():
        filepath = os.path.join(images_dir, name + ".png")
        if not overwrite and os.path.exists(filepath):
            continue
        species.molecule[0].draw(filepath)

def prettydot(species_path, dotfilepath, strip_line_labels=False):
    """
    Make a prettier version of the dot file (flux diagram)

    Assumes the species pictures are stored in a directory
    called 'species_pictures' alongside the dot file.
    """
    
    pictures_directory = f"{species_path}/"

    if strip_line_labels:
        print("stripping edge (line) labels")

    reSize = re.compile('size="5,6"\;page="5,6"')
    reNode = re.compile(
        '(?P<node>s\d+)\ \[\ fontname="Helvetica",\ label="(?P<label>[^"]*)"\]\;'
    )

    rePicture = re.compile("(?P<smiles>.+?)\((?P<id>\d+)\)\.png")
    reLabel = re.compile("(?P<name>.+?)\((?P<id>\d+)\)$")

    species_pictures = dict()
    for picturefile in os.listdir(pictures_directory):
        match = rePicture.match(picturefile)
        if match:
            species_pictures[match.group("id")] = picturefile
        else:
            pass
            # print(picturefile, "didn't look like a picture")

    filepath = dotfilepath

    if not open(filepath).readline().startswith("digraph"):
        raise ValueError("{0} - not a digraph".format(filepath))

    infile = open(filepath)
    prettypath = filepath.replace(".dot", "", 1) + "-pretty.dot"
    outfile = open(prettypath, "w")

    for line in infile:
        (line, changed_size) = reSize.subn('size="12,12";page="12,12"', line)
        match = reNode.search(line)
        if match:
            label = match.group("label")
            idmatch = reLabel.match(label)
            if idmatch:
                idnumber = idmatch.group("id")
                if idnumber in species_pictures:
                    line = (
                        f'%s [ image="{pictures_directory}%s" label="" width="0.5" height="0.5" imagescale=false fixedsize=false color="none" ];\n'
                        % (match.group("node"), species_pictures[idnumber])
                    )

        # rankdir="LR" to make graph go left>right instead of top>bottom
        if strip_line_labels:
            line = re.sub('label\s*=\s*"\s*[\d.]+"', 'label=""', line)

        # change colours
        line = re.sub('color="0.7,\ (.*?),\ 0.9"', r'color="1.0, \1, 0.7*\1"', line)

        outfile.write(line)

    outfile.close()
    infile.close()
#     print(f"Graph saved to: {prettypath}")
    os.system(f'dot {prettypath} -Tpng -o{prettypath.replace(".dot", "", 1) + ".png"} -Gdpi=200')
    return prettypath

def show_flux_diagrams(self, suffix="", embed=False):
    """
    Shows the flux diagrams in the notebook.
    Loads them from disk.
    Does not embed them, to keep the .ipynb file small,
    unless embed=True. Use embed=True if you might over-write the files,
    eg. you want to show flux at different points.
    """
    import IPython

    for element in "CHON":
        for phase_object in (self.gas, self.surf):
            phase = phase_object.name
            img_file = (
                f"reaction_path_{element}_{phase}{'_' if suffix else ''}{suffix}.png"
            )
            display(IPython.display.HTML(f"<hr><h2>{element} {phase}</h2>"))
            if embed:
                display(IPython.display.Image(filename=img_file, width=400, embed=True))
            else:
                display(IPython.display.Image(url=img_file, width=400, embed=False))

        # Now do the combined
        img_file = f"reaction_path_mass{'_' if suffix else ''}{suffix}.png"
        display(IPython.display.HTML(f"<hr><h2>Combined mass</h2>"))
        if embed:
            display(IPython.display.Image(filename=img_file, width=400, embed=True))
        else:
            display(IPython.display.Image(url=img_file, width=400, embed=False))

def save_flux_diagrams(*phases, suffix="", timepoint="", species_path=""):
    """
    Saves the flux diagrams. The filenames have a suffix if provided,
    so you can keep them separate and not over-write.
    """
    for element in "CHON":
        for phase_object in phases:
            phase = phase_object.name

            diagram = ct.ReactionPathDiagram(phase_object, element)
            diagram.title = f"Reaction path diagram following {element} in {phase}"
            diagram.label_threshold = 0.001

            dot_file = f"{suffix}/reaction_path_{element}_{phase}_{timepoint}.dot"
            img_file = f"{suffix}/reaction_path_{element}_{phase}_{timepoint}.png"
            dot_bin_path = (
                "/Users/lee.ting/Code/anaconda3/pkgs/graphviz-2.40.1-hefbbd9a_2/bin/dot" 
                #maybe try "/home/lee.ting/.conda/pkgs/graphviz-2.40.1-h21bd128_2/bin/dot"
            )
            img_path = os.path.join(os.getcwd(), img_file)
            diagram.write_dot(dot_file)

            #also make a prettydot file
            prettydot(species_path, dot_file, strip_line_labels=False)

            # print(diagram.get_data())

#             print(
#                 f"Wrote graphviz input file to '{os.path.join(os.getcwd(), dot_file)}'."
#             )
            os.system(f"dot {dot_file} -Tpng -o{img_file} -Gdpi=200")
#             print(f"Wrote graphviz output file to '{img_path}'.")


In [2]:
#atol_rtol_array = [1e-9, 1e-10,1e-11,1e-12, 1e-13,1e-14,1e-15,1e-16,1e-17,
#                   1e-18,1e-19,1e-20,1e-21,1e-22,1e-23,1e-24,1e-25,1e-26,1e-27]
#rtol_diff = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7,1e-8, 1e-9, 1e-10]
# atol_rtol_array = np.logspace(-29, -9, num=21)
# rtol_diff = np.logspace(1, 10, num=10)

In [3]:
#######################################################################
# Input Parameters for combustor
#######################################################################

# filepath for writing files
notebook_dir = '/work/westgroup/lee.ting/cantera/ammonia_analysis/Troubleshooting/'
git_repo = "/work/westgroup/lee.ting/cantera/ammonia/"
cti_file = git_repo + "base/cantera/chem_annotated.cti"

temp = 587 # kelvin
pressure = 1 * ct.one_atm  # Pascals
volume_flow = 2.7155e-8 # [m^3/s]
#3500 Ncm3/min = 3500/e6/60 m3/s = 5.8333e-5

# NH3/O2 = 0.068
X_o2 = 0.88
x_O2_str = str(X_o2)[0:3].replace(".", "_")
X_nh3 = 0.066 
x_NH3_str = str(X_nh3)[0:8].replace(".", "_")
X_he = 1 - X_o2 - X_nh3

# sensitivity settings
sensitivity = False
sensatol = 1e-6
sensrtol = 1e-6

# Specify idealGasReactor and isothermal
reactor_type=1
energy="off"

# 14 aluminum plates, each of them containing seven semi-cylindrical microchannels of 280 µm width 
# and 140 µm depth, 9 mm long, arranged at equal distances of 280 µm 

# get git commit hash and message
rmg_model_path = "../../ammonia"
repo = git.Repo(rmg_model_path)
date = time.localtime(repo.head.commit.committed_date)
git_date = f"{date[0]}_{date[1]}_{date[2]}_{date[3]}{date[4]}"
git_sha = str(repo.head.commit)[0:6]
git_msg = str(repo.head.commit.message)[0:50].replace(" ", "_").replace("'", "_").replace("\n", "")
git_file_string = f"{git_date}_{git_sha}_{git_msg}"

# set sensitivity string for file path name
if sensitivity:
    sensitivity_str = "on"
else: 
    sensitivity_str = "off"


# constants
pi = math.pi

# set initial temps, pressures, concentrations
temp = 700 # kelvin

pressure = 1 * ct.one_atm  # Pascals

surf_temp = temp



mw_nh3 = 17.0306e-3  # [kg/mol]
mw_o2 = 31.999e-3  # [kg/mol]
mw_he = 4.002602e-3  # [kg/mol]

o2_ratio = X_nh3 / X_o2

# O2/NH3/He: typical is
concentrations_rmg = {"O2(2)": X_o2, "NH3(6)": X_nh3, "He": X_he}

# initialize cantera gas and surface
gas = ct.Solution(cti_file, "gas")
surf = ct.Interface(cti_file, "surface1", [gas])

# initialize temperatures 
gas.TPX = temp, pressure, concentrations_rmg
surf.TP = temp, pressure # change this to surf_temp when we want a different starting temperature for the surface

# if a mistake is made with the input, 
# cantera will normalize the mole fractions. 
# make sure that we are reporting/using 
# the normalized values
X_o2 = float(gas["O2(2)"].X)
X_nh3 = float(gas["NH3(6)"].X)
X_he = float(gas["He"].X)

# create gas inlet
inlet = ct.Reservoir(gas)

# create gas outlet
exhaust = ct.Reservoir(gas)

# Reactor volume
number_of_reactors = 1001
rradius = 1.4e-4 #140µm to 0.00014m
rtotal_length = 9e-3 #9mm to 0.009m
rtotal_vol = (rradius ** 2) * pi * rtotal_length / 2

rlength = rtotal_length/1000


# divide totareactor total volume 
rvol = (rtotal_vol )/number_of_reactors


# Catalyst Surface Area
site_density = (surf.site_density * 1000)  # [mol/m^2] cantera uses kmol/m^2, convert to mol/m^2
cat_area_total = rradius * 2 / 2 * pi * rtotal_length # [m^3]
cat_area = cat_area_total / number_of_reactors

# reactor initialization
if reactor_type == 0:
    r = ct.Reactor(gas, energy=energy)
    reactor_type_str = "Reactor"
elif reactor_type == 1:
    r = ct.IdealGasReactor(gas, energy=energy)
    reactor_type_str = "IdealGasReactor"
elif reactor_type == 2:
    r = ct.ConstPressureReactor(gas, energy=energy)
    reactor_type_str = "ConstPressureReactor"
elif reactor_type == 3:
    r = ct.IdealGasConstPressureReactor(gas, energy=energy)
    reactor_type_str = "IdealGasConstPressureReactor"

# calculate the available catalyst area in a differential reactor
rsurf = ct.ReactorSurface(surf, r, A=cat_area)
r.volume = rvol
surf.coverages = "X(1):1.0"

# flow controllers 
one_atm = ct.one_atm
FC_temp = 293.15

molar_flow = volume_flow * one_atm / (8.3145 * FC_temp)  # [mol/s]
mass_flow = molar_flow * (X_nh3 * mw_nh3 + X_o2 * mw_o2 + X_he * mw_he)  # [kg/s]
mfc = ct.MassFlowController(inlet, r, mdot=mass_flow)

# A PressureController has a baseline mass flow rate matching the 'master'
# MassFlowController, with an additional pressure-dependent term. By explicitly
# including the upstream mass flow rate, the pressure is kept constant without
# needing to use a large value for 'K', which can introduce undesired stiffness.
outlet_mfc = ct.PressureController(r, exhaust, master=mfc, K=0.01)

# initialize reactor network
sim = ct.ReactorNet([r])

# set relative and absolute tolerances on the simulation
sim.rtol = 1e-9
sim.atol = 1e-22

surf.advance_coverages(10)

#################################################
# Run single reactor
#################################################

# round numbers for filepath strings so they're easier to read
cat_area_str = "%s" % "%.3g" % cat_area

# if it doesn't already exist, create
species_path = (
    notebook_dir 
    + f"/species_pictures"
)

results_path = (
    notebook_dir  
    + f"/results"
)

flux_path = (
    notebook_dir  
    + f"/flux_diagrams"
)
# create species folder for results if they don't exist already
try:
    os.makedirs(species_path, exist_ok=True)
    save_pictures(git_path=rmg_model_path, species_path=species_path)
except OSError as error:
    print(error)

try:
    os.makedirs(results_path, exist_ok=True)
except OSError as error:
    print(error)

try:
    os.makedirs(flux_path, exist_ok=True)
except OSError as error:
    print(error)

gas_ROP_str = [i + " ROP [kmol/m^3 s]" for i in gas.species_names]

# surface ROP reports gas and surface ROP. these values are not redundant
gas_surf_ROP_str = [i + " surface ROP [kmol/m^2 s]" for i in gas.species_names]
surf_ROP_str = [i + " ROP [kmol/m^2 s]" for i in surf.species_names]

gasrxn_ROP_str = [i + " ROP [kmol/m^3 s]" for i in gas.reaction_equations()]
surfrxn_ROP_str = [i + " ROP [kmol/m^2 s]" for i in surf.reaction_equations()]

output_filename = (
    results_path
    + f"/Spinning_basket_results.csv"
)

outfile = open(output_filename, "w")
writer = csv.writer(outfile)

# Sensitivity atol, rtol, and strings for gas and surface reactions if selected
# slows down script by a lot
if sensitivity:
    sim.rtol_sensitivity = sensrtol
    sim.atol_sensitivity = sensatol
    sens_species = ["NH3(6)", "O2(2)", "N2(4)", "NO(5)", "N2O(7)"]  #change THIS to your species, can add "," and other species

    # turn on sensitive reactions
    for i in range(gas.n_reactions):
        r.add_sensitivity_reaction(i)

    for i in range(surf.n_reactions):
        rsurf.add_sensitivity_reaction(i)
    
    # create column names for sensitivity data
    for j in sens_species:
        gasrxn_sens_str = [
            j + " sensitivity to " + i for i in gas.reaction_equations()
        ]
        surfrxn_sens_str = [
            j + " sensitivity to " + i for i in surf.reaction_equations()
        ]
        sens_list = gasrxn_sens_str + surfrxn_sens_str  # + gastherm_sens_str

    writer.writerow(
        [
            "Distance (mm)",
            "T (C)",
            "P (Pa)",
            "V (M^3/s)",
            "X_nh3 initial", 
            "X_o2 initial",
            "X_he initial",
            "(NH3/O2)",
            "T (C) final",
            "Rtol",
            "Atol",
            "reactor type",
            "energy on?"
        ]
        + gas.species_names
        + surf.species_names
        + gas_ROP_str
        + gas_surf_ROP_str
        + surf_ROP_str
        + gasrxn_ROP_str
        + surfrxn_ROP_str
        + sens_list
    )

else:

    writer.writerow(
        [
            "Distance (mm)",
            "T (C)",
            "P (Pa)",
            "V (M^3/s)",
            "X_nh3 initial",
            "X_o2 initial",
            "X_he initial",
            "(NH3/O2)",
            "T (C) final",
            "Rtol",
            "Atol",
            "reactor type",
            "energy on?"
        ]
        + gas.species_names
        + surf.species_names
        + gas_ROP_str
        + gas_surf_ROP_str
        + surf_ROP_str
        + gasrxn_ROP_str
        + surfrxn_ROP_str
    )

# initialize time and iteration count, set time step
t = 0.0
dt = 0.1
iter_ct = 0

# run the simulation
first_run = True
distance_mm = 0

for n in range(number_of_reactors):


    # Set the state of the reservoir to match that of the previous reactor
    gas.TDY = TDY = r.thermo.TDY
    inlet.syncState()
    sim.reinitialize()
    previous_coverages = surf.coverages  # in case we want to retry
    
    # Add a first row in the CSV with just the feed
    writer.writerow(
        [
            distance_mm,
            temp,
            gas.P,
            volume_flow,
            X_nh3,
            X_o2,
            X_he,
            o2_ratio,
            gas.T,
            sim.rtol,
            sim.atol,
            reactor_type_str,
            energy,
        ]
        + list(gas.X)
        + list(surf.X)
        + list(gas.net_production_rates)
        + list(surf.net_production_rates)
        + list(gas.net_rates_of_progress)
        + list(surf.net_rates_of_progress)
    )
    if n > 0:  
        try:
            sim.advance_to_steady_state(return_residuals=True)
        except ct.CanteraError:
            t = sim.time
            sim.set_initial_time(0)
            gas.TDY = TDY
            surf.coverages = previous_coverages
            r.syncState()
            sim.reinitialize()
            new_target_time = 0.01 * t
            logging.warning(
                f"Couldn't reach {t:.1g} s so going to try {new_target_time:.1g} s"
            )
            try:
                sim.advance(new_target_time)
            except ct.CanteraError:
                outfile.close()
                raise

    # save flux diagrams at beginning of run
    if first_run == True:
        save_flux_diagrams(gas, suffix=flux_path, timepoint="beginning", species_path=species_path)
        save_flux_diagrams(surf, suffix=flux_path, timepoint="beginning", species_path=species_path)
        first_run = False

    if sensitivity:
        # get sensitivity for sensitive species i (e.g. methanol) in reaction j
        for i in sens_species:
            g_nrxn = gas.n_reactions
            s_nrxn = surf.n_reactions

            gas_sensitivities = [sim.sensitivity(i, j) for j in range(g_nrxn)]
            surf_sensitivities = [
                sim.sensitivity(i, j) for j in range(g_nrxn, g_nrxn + s_nrxn)
            ]

            sensitivities_all = (
                gas_sensitivities
                + surf_sensitivities
            )

        writer.writerow(
            [
                distance_mm,
                temp,
                gas.P,
                volume_flow,
                X_nh3,
                X_o2,
                X_he,
                o2_ratio,
                gas.T,
                sim.rtol,
                sim.atol,
                reactor_type_str,
                energy,
            ]
            + list(gas.X)
            + list(surf.X)
            + list(gas.net_production_rates)
            + list(surf.net_production_rates)
            + list(gas.net_rates_of_progress)
            + list(surf.net_rates_of_progress)
            + sensitivities_all,
        )

    else:
        writer.writerow(
            [
                distance_mm,
                temp,
                gas.P,
                volume_flow,
                X_nh3,
                X_o2,
                X_he,
                o2_ratio,
                gas.T,
                sim.rtol,
                sim.atol,
                reactor_type_str,
                energy,
            ]
            + list(gas.X)
            + list(surf.X)
            + list(gas.net_production_rates)
            + list(surf.net_production_rates)
            + list(gas.net_rates_of_progress)
            + list(surf.net_rates_of_progress)
        )


    iter_ct += 1
    distance_mm = n * rlength * 1.0e3  # distance in mm

outfile.close()

# save flux diagrams at the end of the run
save_flux_diagrams(gas, suffix=flux_path, timepoint="end", species_path=species_path)
save_flux_diagrams(surf, suffix=flux_path, timepoint="end", species_path=species_path)
#print(f"passed! atol={atol_rtol_array}")




CanteraError: 
***********************************************************************
CanteraError thrown by CVodesIntegrator::integrate:
CVodes error encountered. Error code: -3
At t = 1.7518e-06 and h = 5.32702e-17, the error test failed repeatedly or with |h| = hmin.

Exceptions caught during RHS evaluation:

***********************************************************************
CanteraError thrown by Phase::setDensity():
density must be positive. density = -6.09241
***********************************************************************


***********************************************************************
CanteraError thrown by Phase::setDensity():
density must be positive. density = -19.1056
***********************************************************************


***********************************************************************
CanteraError thrown by Phase::setDensity():
density must be positive. density = -43.9051
******************************************************

In [21]:
sim.component_name(15)

'IdealGasReactor_8: HNO(103)'

H2OX <=> H2O + X

In [4]:
type(surf.forward_rate_constants)


numpy.ndarray

In [4]:
for i,j in enumerate(surf.reaction_equations()):
    if "N2O(7)" in j:
        print(j, "rate constant: ", "{:.2e}".format(surf.forward_rate_constants[i]))
        

NO_X(14) + N_X(13) <=> N2O(7) + 2 X(1) rate constant:  4.21e+10
N2O(7) + X(1) <=> N2(4) + O_X(9) rate constant:  3.84e+02
N2O_X(51) <=> N2O(7) + X(1) rate constant:  1.40e+16
N2O(7) + X(1) <=> N2O_X(48) rate constant:  5.84e+09


In [5]:
for i,j in enumerate(surf.reaction_equations()):
    if "N2(4)" in j:
        print(j, "rate constant: ", "{:.2e}".format(surf.forward_rate_constants[i]))

2 N_X(13) <=> N2(4) + 2 X(1) rate constant:  1.80e+11
N2O(7) + X(1) <=> N2(4) + O_X(9) rate constant:  3.84e+02
N2(4) + X(1) <=> N2X(199) rate constant:  4.03e+05


In [6]:
surf_rates = {}
for i,j in enumerate(surf.reaction_equations()):
    surf_rates[j]=surf.forward_rate_constants[i]
{k: v for k, v in sorted(surf_rates.items(), key=lambda item: item[1], reverse=True)}


{'NH2_X(10) + O_X(9) <=> NH_X(12) + OH_X(11)': 4.03e+20,
 'HONO(190) + X(1) <=> HNOX(189) + O_X(9)': 1.2123981697290792e+20,
 'HX(18) + O_X(9) <=> OH_X(11) + X(1)': 1.8682268601059308e+19,
 'NHO_X(15) + OH_X(11) <=> H2O_X(16) + NO_X(14)': 1.9824552780396447e+18,
 'NHO_X(15) + OH_X(11) <=> HONO(190) + HX(18)': 1.9824552780396447e+18,
 'NHO_X(15) + NO2_X(28) <=> HONO(190) + NO_X(14)': 1.9824552780396447e+18,
 '2 NH2_X(10) <=> NH3_X(8) + NH_X(12)': 5.2728514218645965e+17,
 'HONO(190) + HX(18) <=> H2O_X(16) + NO_X(14)': 2.6880982733361658e+17,
 'NH_X(12) + OH_X(11) <=> H2O_X(16) + N_X(13)': 1.4889913527758058e+17,
 'HONO(190) + OH_X(11) <=> H2O_X(16) + NO2_X(28)': 4.030780984706911e+16,
 'HONO(190) + X(1) <=> NO_X(14) + OH_X(11)': 2.3599296913986084e+16,
 'NH2_X(10) + NHO_X(15) <=> NH3_X(8) + NO_X(14)': 2.221503351723133e+16,
 'HONO(190) + NH2_X(10) <=> NH3_X(8) + NO2_X(28)': 2.221503351723133e+16,
 'HX(18) + NH2_X(10) <=> NH3_X(8) + X(1)': 2.103501570623314e+16,
 'NH_X(12) + O_X(9) <=> N_

In [7]:
gas_rates = {}
for i,j in enumerate(gas.reaction_equations()):
    gas_rates[j]=gas.forward_rate_constants[i]
{k: v for k, v in sorted(gas_rates.items(), key=lambda item: item[1], reverse=True)}


{'H2NO(104) + OH(33) <=> H2O(3) + HNO(103)': 100000000000.0,
 'HNO2(110) + OH(33) <=> H2O(3) + NO2(29)': 40000000000.0,
 'HO2(53) + NH2(37) <=> H2NO(104) + OH(33)': 24670627104.832893,
 'NO3(111) + OH(33) <=> HO2(53) + NO2(29)': 14000000000.0,
 'HO2(53) + OH(33) <=> H2O(3) + O2(2)': 10377555661.944689,
 'NH2(37) + NO2(29) <=> H2NO(104) + NO(5)': 4147035190.5375533,
 'HO2(53) + NO(5) <=> NO2(29) + OH(33)': 3001833401.3324156,
 'NH2(37) + NO(5) <=> H2O(3) + N2(4)': 2531134830.1997747,
 'HONO(107) + OH(33) <=> H2O(3) + NO2(29)': 2470569026.5850644,
 'HNO(103) + OH(33) <=> H2O(3) + NO(5)': 2278900694.665231,
 'HO2(53) + NH2(37) <=> H2O(3) + HNO(103)': 2231691276.710161,
 'HNO(103) + NH2(37) <=> NH3(6) + NO(5)': 1765794614.8256946,
 'HO2(53) + NO3(111) <=> NO2(29) + O2(2) + OH(33)': 1500000000.0,
 'H2NO(104) + NH2(37) <=> HNO(103) + NH3(6)': 1461885726.77501,
 'NH2(37) + NO2(29) <=> H2O(3) + N2O(7)': 1060869467.346816,
 'HONO(107) + NH2(37) <=> NH3(6) + NO2(29)': 967745374.3485787,
 'NH3(6)

In [8]:
gas_equilib = {}
for i,j in enumerate(gas.reaction_equations()):
    gas_equilib[j]=gas.equilibrium_constants[i]
{k: v for k, v in sorted(gas_equilib.items(), key=lambda item: item[1], reverse=True)}


{'NH2(37) + NO(5) <=> H2O(3) + N2(4)': 5.285829878442484e+37,
 'NH2(37) + NO2(29) <=> H2O(3) + N2O(7)': 1.1164419397686727e+27,
 '2 HNO(103) <=> H2O(3) + N2O(7)': 1.2391640063399868e+26,
 'HO2(53) + NH2(37) <=> H2O(3) + HNO(103)': 1.3622563170127475e+24,
 'HNO(103) + OH(33) <=> H2O(3) + NO(5)': 2.1485575781203113e+21,
 'HO2(53) + OH(33) <=> H2O(3) + O2(2)': 4.863560474363e+20,
 'HNO(103) + NH2(37) <=> NH3(6) + NO(5)': 2.615804147376036e+17,
 'HO2(53) + NH2(37) <=> NH3(6) + O2(2)': 5.9212384110194856e+16,
 'H2NO(104) + OH(33) <=> H2O(3) + HNO(103)': 1.1411074734049808e+16,
 'HNO2(110) + OH(33) <=> H2O(3) + NO2(29)': 1034980709872743.4,
 'H2NO(104) + NH2(37) <=> HNO(103) + NH3(6)': 1389263984326.5972,
 'HONO(107) + OH(33) <=> H2O(3) + NO2(29)': 1300761226928.0972,
 'N2O(7) + NO(5) <=> N2(4) + NO2(29)': 47345318105.28139,
 'HNO(103) + NO2(29) <=> HONO(107) + NO(5)': 1651769389.832121,
 'N2O(7) + OH(33) <=> HO2(53) + N2(4)': 672779202.1833456,
 'HO2(53) + NO2(29) <=> HONO(107) + O2(2)': 37

In [9]:
surf_equilib = {}
for i,j in enumerate(surf.reaction_equations()):
    surf_equilib[j]=surf.equilibrium_constants[i]
{k: v for k, v in sorted(surf_equilib.items(), key=lambda item: item[1], reverse=True)}


{'N2_X(17) + X(1) <=> N2X2(77)': 1.0110266683858243e+151,
 'N2_X(17) + O2_ads(48) <=> N2OXX(54) + O_X(9)': 1.1109923215624261e+142,
 'N2_X(17) + NOX2(52) <=> N2OXX(54) + N_X(13)': 8.171949790232787e+104,
 'N2O2(79) + 2 X(1) <=> 2 NO_X(14)': 2.0399912999220073e+82,
 'O2(2) + 2 X(1) <=> 2 O_X(9)': 1.8011602557882756e+57,
 'N2O2(79) + 2 X(1) <=> NNOJJX(78) + O_X(9)': 1.2774297816139405e+53,
 'HONO(64) + HX(20) <=> H2O_X(18) + NO_X(14)': 5.0852764247382086e+44,
 'N2O_X(16) + X(1) <=> N2OXX(54)': 7.1197409527219e+42,
 'NOX2(52) + N_X(13) <=> N2X2(77) + O_X(9)': 2.4428293797687812e+42,
 'HO2X(50) + X(1) <=> OH_X(11) + O_X(9)': 2.5287191541353727e+38,
 'OH(D)(31) + X(1) <=> OH_X(11)': 2.3701832052057948e+38,
 'N_X(13) + O2_ads(48) <=> NOX2(52) + O_X(9)': 1.359519270285104e+37,
 'NHO_X(15) + OH_X(11) <=> H2O_X(18) + NO_X(14)': 2.000129518337332e+36,
 'HONO(62) + 2 X(1) <=> NO_X(14) + OH_X(11)': 3.2957303797297498e+35,
 'N2O(7) + 2 X(1) <=> N2OXX(54)': 8.338447296778552e+32,
 'N2O_X(16) + X(1) 

In [9]:
surf_rates = {}
for i,j in enumerate(surf.reaction_equations()):
    surf_rates[j]=surf.reverse_rate_constants[i]
{k: v for k, v in sorted(surf_rates.items(), key=lambda item: item[1], reverse=True)}


{'H2O_X(16) + O_X(9) <=> 2 OH_X(11)': 1.62085883605505e+23,
 'H2O(3) + 2 X(1) <=> HX(18) + OH_X(11)': 1.1441200426111358e+22,
 'NH3(6) + 2 X(1) <=> HX(18) + NH2_X(10)': 5.681575178028749e+20,
 'H2O_X(16) + N_X(13) <=> HNOX(189) + HX(18)': 4.93138683536746e+20,
 'NH2_X(10) + O_X(9) <=> NH_X(12) + OH_X(11)': 4.8370205259951106e+20,
 'HONO(190) + X(1) <=> HNOX(189) + O_X(9)': 1.4045470421592747e+20,
 'NH_X(12) + O_X(9) <=> NHO_X(15) + X(1)': 1.213090953473967e+20,
 'NO_X(14) + N_X(13) <=> N2O_X(48) + X(1)': 1.1389478102885582e+20,
 'HX(18) + O_X(9) <=> OH_X(11) + X(1)': 1.4633792820175872e+19,
 'NH_X(12) + O_X(9) <=> N_X(13) + OH_X(11)': 4.3993254451565536e+17,
 'NHO_X(15) + OH_X(11) <=> HONO(190) + HX(18)': 4.261811430450053e+17,
 'NO_X(14) + O_X(9) <=> NO2_X(28) + X(1)': 9.173509814583426e+16,
 'NH3_X(8) + O_X(9) <=> NH2_X(10) + OH_X(11)': 2.1491371213484972e+16,
 'N2O_X(51) <=> N2O(7) + X(1)': 9200593072842486.0,
 'HONO(190) + X(1) <=> HX(18) + NO2_X(28)': 927832616556325.6,
 'HONO(190

In [15]:
gas_rates = {}
for i,j in enumerate(gas.reaction_equations()):
    gas_rates[j]=gas.reverse_rate_constants[i]
{k: v for k, v in sorted(gas_rates.items(), key=lambda item: item[1], reverse=True)}


{'2 NO2(29) <=> NO(5) + NO3(111)': 79272536009.8445,
 'H2NO(104) + NO(5) <=> 2 HNO(103)': 104282182.29643598,
 'HO2(53) + NO(5) <=> NO2(29) + OH(33)': 18536331.754097104,
 'H2NO(104) + O2(2) <=> HNO(103) + HO2(53)': 7678644.1253331695,
 'NH3(6) + OH(33) <=> H2O(3) + NH2(37)': 17643.494380106506,
 '2 NO2(29) <=> 2 NO(5) + O2(2)': 8854.828230059871,
 'HNO(103) + O2(2) <=> HO2(53) + NO(5)': 6013.222062023728,
 'NO3(111) + OH(33) <=> HO2(53) + NO2(29)': 714.4467216218208,
 'NH2(37) + NO2(29) <=> H2NO(104) + NO(5)': 471.90340478852596,
 'H2NO(104) + NO2(29) <=> HNO(103) + HONO(107)': 424.70911349168426,
 'HO2(53) + NO3(111) <=> NO2(29) + O2(2) + OH(33)': 62.028431754880174,
 'N2O(7) + OH(33) <=> HNO(103) + NO(5)': 45.467779710793614,
 'NO2(29) + NO3(111) <=> NO(5) + NO2(29) + O2(2)': 29.980897705269967,
 'HO2(53) + NH2(37) <=> H2NO(104) + OH(33)': 15.215529651807607,
 '2 HONO(107) <=> H2O(3) + NO(5) + NO2(29)': 1.5175621749973196,
 'HONO(107) + NH2(37) <=> NH3(6) + NO2(29)': 0.1848513638341

In [16]:
sim.component_name(14)

'IdealGasReactor_5: HO2(53)'

In [13]:
import pandas as pd

In [14]:
pd.read_csv('/work/westgroup/lee.ting/cantera/ammonia_analysis/2021_6_5_1819_463449_master_523K_O088N006/IdealGasReactor/energy_off/sensitivity_off/415/results/Spinning_basket_area_3.95e-09_energy_off_temp_415_O2_0_1_NH3_0_01.csv#')

FileNotFoundError: [Errno 2] No such file or directory: '/work/westgroup/lee.ting/cantera/ammonia_analysis/2021_6_5_1819_463449_master_523K_O088N006/IdealGasReactor/energy_off/sensitivity_off/415/results/Spinning_basket_area_3.95e-09_energy_off_temp_415_O2_0_1_NH3_0_01.csv#'